<a href="https://colab.research.google.com/github/quantasaurus/strategy_backtest_long/blob/main/backtester.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance -q

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf


In [ ]:
# Currently backtesting strategy on HDL
# Other algorand related assets include: YLDY-USD, OPUL-USD, ALGO-USD

asset = "HDL-USD"
df = yf.download(asset, start="2022-06-15", end="2022-12-15", interval='1h')[['Close']]
df

In [ ]:
# Here we can specify our moving average intervals.  50 and 200 are commonly used in trading
SMA_1 = 50
SMA_2 = 200

df[f'{SMA_1}_MA'] = df['Close'].rolling(SMA_1).mean()
df[f'{SMA_2}_MA'] = df['Close'].rolling(SMA_2).mean()

In [ ]:
df.plot(figsize=(16,9))

In [ ]:
# Calculate log returns of close price to use in profitability calculation 
df['returns'] = np.log(df['Close'] / df['Close'].shift(1))

In [ ]:
# Bot buys when SMA_1 < SMA_2, then sells when SMA_1 > SMA_2
df['position'] = np.where(df[f'{SMA_1}_MA'] < df[f'{SMA_2}_MA'], 1, 0)

In [ ]:
df[['Close', f'{SMA_1}_MA', f'{SMA_2}_MA']].plot(figsize=(16,9))
df['position'].plot(secondary_y=True)

In [ ]:
df['strategy_returns'] = df['returns'] * df['position'].shift(1)
df

In [ ]:
df[['Close', f'{SMA_1}_MA', f'{SMA_2}_MA']].plot(figsize=(16,9))
df['strategy_returns'].cumsum().apply(np.exp).plot(secondary_y=True, legend=True)

In [ ]:
df[['returns', 'strategy_returns']].cumsum().apply(np.exp).plot(figsize=(16,9), title='Strategy Returns vs Market Returns')